In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options,ChromiumOptions
from selenium.webdriver import EdgeOptions
import xlwings as xw

options = EdgeOptions()
options.add_argument("--headless")

driver = webdriver.ChromiumEdge(options=options)
wb=xw.Book("E:\code\java\QuanLyThuVien\Book.xlsx")
sheet=wb.sheets.active
sheet.range("A1").value='Trang'
sheet.range("B1").value='Image Sách'
sheet.range("C1").value='Tên Sách'
sheet.range("D1").value='Nhà Xuất Bản'
sheet.range("E1").value='Năm Xuất Bản'
sheet.range("F1").value='Tác Giả'
sheet.range("G1").value='Số Trang'
sheet.range("H1").value='Nội Dung'

i=2003
row=45193
while(i<=2005):
    sheet.range(f"A{row}").value=i
    url = 'https://www.fahasa.com/sach-trong-nuoc.html?order=created_at&limit=24&p='+ str(i)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # Tìm tất cả các thẻ <div> có class="product-item"
    product_items = soup.find_all('div', class_="item-inner")# item_inner
    for product_item in product_items:
        if(product_item.find('h2', class_='product-name-no-ellipsis p-name-list')==None):
            continue
        # Tìm hình ảnh sách
        # Tìm tiêu đề sách
        title = product_item.find('h2', class_='product-name-no-ellipsis p-name-list').find('a')['title'] if product_item.find('h2', class_='product-name-no-ellipsis p-name-list').find('a')['title']!=None else "????"
        # Tìm url child
        url_child=product_item.find('h2', class_='product-name-no-ellipsis p-name-list').find('a')['href']
        driver.get(url_child)
        soup_child = BeautifulSoup(driver.page_source, 'html.parser')
        image = soup_child.find('img', class_='swiper-lazy swiper-lazy-loaded')['src'] if soup_child.find('img', class_='swiper-lazy swiper-lazy-loaded') !=None else "????"
        nxb=soup_child.find('td',class_='data_publisher').text.strip() if soup_child.find('td',class_='data_publisher') !=None else "????"

        nam_xb=soup_child.find('td',class_='data_publish_year').text.strip() if soup_child.find('td',class_='data_publish_year')!=None else "????"
        
        author=soup_child.find('td',class_='data_author').text.strip() if soup_child.find('td',class_='data_author') !=None else "????"

        num_of_page=soup_child.find('td',class_='data_qty_of_page').text.strip() if soup_child.find('td',class_='data_qty_of_page')!=None else "????"
        
        contents=soup_child.find('div',class_='std').findAll('p') if soup_child.find('div',class_='std')!=None else ["????"]
        arr_content=""
        for content in contents:
            arr_content+=f"{content.text.strip()}\n" if soup_child.find('div',class_='std')!=None else f"{content}\n"
        
        sheet.range(f"B{row}").value=image
        sheet.range(f"C{row}").value=title
        sheet.range(f"D{row}").value=nxb
        sheet.range(f"E{row}").value=nam_xb
        sheet.range(f"F{row}").value=author
        sheet.range(f"G{row}").value=num_of_page
        sheet.range(f"H{row}").value=arr_content
        sheet.range(f"H{row}").row_height=16
        row+=1
    i+=1
    wb.save("E:\code\java\QuanLyThuVien\Book.xlsx")
